In [ ]:
def filter_dim_rois(fluo, sigma_thresh=2):
    """Remove dim ROIs based on mean fluorescence."""
    mean_f = np.mean(fluo, axis=0)
    thresh = np.mean(mean_f) - sigma_thresh * np.std(mean_f)
    mask = mean_f >= thresh
    removed = np.where(~mask)[0]
    kept = np.where(mask)[0]
    fluo_filtered = fluo[:, mask]

    print(f"Removed {len(removed)} dim ROIs: {removed.tolist()}")
    return fluo_filtered, kept, removed

def compute_percentile_baseline(fluo, fps, tau, percentile=8, instability_ratio=0.1):
    """Compute F0 baseline and discard unstable ROIs."""
    T, N = fluo.shape
    w = int(max(15, 40 * tau) * fps)
    F0 = np.full_like(fluo, np.nan)
    unstable_indices = []

    for n in range(N):
        trace = fluo[:, n]
        baseline = np.zeros_like(trace)
        for t in range(w, T - w):
            baseline[t] = np.percentile(trace[t - w:t + w + 1], percentile)
        baseline[:w] = baseline[w]
        baseline[-w:] = baseline[-w - 1]
        if np.min(baseline) < instability_ratio * np.max(baseline):
            unstable_indices.append(n)
            continue
        F0[:, n] = uniform_filter1d(baseline, size=w)
    return F0, unstable_indices

def filter_inactive_rois_by_std(deltaF_F, final_kept, final_removed, min_std=0.01, verbose=True):
    """
    Remove ROIs with low std in ΔF/F.
    """
    stds = np.std(deltaF_F, axis=0)
    mean_std = np.mean(stds)  # compute mean of stds
    active_mask = stds > min_std
    inactive_indices = np.where(~active_mask)[0]
    deltaF_F_active = deltaF_F[:, active_mask]

    # Map inactive indices back to original fluo_raw indexing
    activity_removed = final_kept[inactive_indices]

    # Update removed and kept indices in fluo_raw space
    final_removed_all = np.sort(np.unique(np.concatenate([final_removed, activity_removed])))
    final_kept_active = final_kept[active_mask]

    if verbose:
        print(f"Mean std across all ROIs: {mean_std:.5f}")
        print(f"Removed {len(inactive_indices)} inactive ROIs (std < {min_std})")
        print(f"Remaining: {deltaF_F_active.shape[1]} ROIs")
        print(f"Total removed (cumulative): {len(final_removed_all)}")

    return deltaF_F_active, final_kept_active, final_removed_all

def compute_deltaF_F(
    fluo_raw,
    fps,
    tau,
    percentile=8,
    instability_ratio=0.1,
    sigma_thresh=2,
    min_std=0.01,
    verbose=True
):
    # Step 1: Filter dim ROIs
    fluo_filtered, kept_rois, dim_removed = filter_dim_rois(fluo_raw, sigma_thresh)

    # Step 2: Compute baseline on filtered data
    F0, unstable_local = compute_percentile_baseline(fluo_filtered, fps, tau, percentile, instability_ratio)

    # Step 3: Determine which of the dim-kept ROIs are unstable
    unstable_mask = np.isnan(F0).all(axis=0)
    unstable_indices = np.where(unstable_mask)[0]
    print(f"Removed {len(unstable_indices)} unstable ROIs: {unstable_indices.tolist()}")

    # Final kept ROIs are those that are both bright enough and stable
    final_kept = kept_rois[~unstable_mask]
    final_removed = np.sort(np.concatenate([dim_removed, kept_rois[unstable_mask]]))

    # Step 4: Keep only stable ROIs
    fluo_clean = fluo_filtered[:, ~unstable_mask]
    F0_clean = F0[:, ~unstable_mask]
    F0_safe = np.where(F0_clean == 0, np.finfo(float).eps, F0_clean)
    deltaF_F = (fluo_clean - F0_clean) / F0_safe

    # Step 5: Remove low-activity ROIs
    deltaF_F_active, final_kept_active, final_removed_all = filter_inactive_rois_by_std(
        deltaF_F, final_kept, final_removed, min_std, verbose
    )


    print(f"ΔF/F₀ computed. Final number of ROIs: {deltaF_F.shape[1]}")

    return deltaF_F_active, final_kept_active, final_removed_all



In [ ]:
data=np.load("C:/Users/suribear/OneDrive - Université de Lausanne/Lab/Data/2p/F.npy").T

deltaF_F, ind_kept, ind_removed = compute_deltaF_F(
    fluo_raw=data,
    fps=2.0,
    tau=3.0,
    percentile=8,
    instability_ratio=0.1,
    sigma_thresh=2,
    min_std=0.01
)


In [ ]:
def gaussfit_neg(data, peak_threshold_fraction=0.2, trim_std=2.0):
    """
    Robust Gaussian fitting of the negative half of a KDE of the input data.

    This method is especially suitable for estimating the noise characteristics
    of ΔF/F traces from calcium imaging, where noise is assumed to dominate
    the negative side of the distribution.

    Parameters:
    -----------
    data : ndarray
        1D array of ΔF/F values for a single ROI over time.
    peak_threshold_fraction : float, optional (default=0.2)
        Fraction of the KDE peak height to include in the fit.
        Values below this threshold are excluded from the Gaussian fit.
    trim_std : float, optional (default=2.0)
        If the fit fails, fallback uses data within ±`trim_std` standard deviations
        to compute robust sigma.

    Returns:
    --------
    sigma : float
        Estimated standard deviation of the noise.
    mu : float
        Estimated mean of the Gaussian fit.
    A : float or np.nan
        Estimated amplitude of the Gaussian. If fallback is used, A is np.nan.
    used_fallback : bool
        True if the fallback method was used due to poor fit.
    """

    N = len(data)

    # Step 1: Estimate the KDE of the input data
    kde = gaussian_kde(data)
    x = np.linspace(np.min(data), np.max(data), 200)
    y = kde(x)

    # Step 2: Find the mode of the distribution (peak of KDE)
    ind_peak = np.argmax(y)

    # Step 3: Use only the left half of the distribution (assumed to be noise-dominated)
    x_fit = x[:ind_peak + 1]
    y_fit = y[:ind_peak + 1]

    # Keep only the part of the curve above a fraction of the peak
    ymax = np.max(y_fit)
    keep = y_fit > ymax * peak_threshold_fraction
    x_fit = x_fit[keep]
    y_fit = y_fit[keep] / N  # normalize to probability density

    # Abort if insufficient data to fit
    if len(x_fit) < 3:
        return np.nan, np.nan, np.nan, True

    try:
        # Step 4: Fit log(y) = a2 x^2 + a1 x + a0
        ylog = np.log(y_fit)
        coeffs = np.polyfit(x_fit, ylog, 2)
        A2, A1, A0 = coeffs

        # Validate parabola direction
        if A2 >= 0:
            raise ValueError("Parabola opens upwards — not a valid Gaussian")

        sigma = np.sqrt(-1 / (2 * A2))
        mu = A1 * sigma ** 2
        A = np.exp(A0 + mu ** 2 / (2 * sigma ** 2))

        if not np.isreal(sigma):
            raise ValueError("Sigma is complex")

        return float(sigma), float(mu), float(A), False  # fit successful

    except Exception:
        # Fallback: use robust std within ±trim_std
        dev = np.nanstd(data)
        mask = np.abs(data) <= trim_std * dev
        trimmed = data[mask]
        sigma = np.nanstd(trimmed)
        mu = np.nanmean(trimmed)
        return float(sigma), float(mu), np.nan, True  # fallback used

def fit_noise_and_center_traces(deltaF_F, fit_function):
    """
    Apply a Gaussian fit to each ROI's ΔF/F trace (typically using only the negative half)
    to estimate noise parameters and center the traces.

    Parameters
    ----------
    deltaF_F : ndarray of shape (T, N)
        Raw ΔF/F data with T timepoints and N ROIs (neurons).
    fit_function : callable
        Function that takes a 1D ΔF/F trace and returns (sigma, mu, A).
        For example: `gaussfit_neg(trace)`.

    Returns
    -------
    deltaF_centered : ndarray of shape (T, N)
        ΔF/F data with mean noise offset (mu) subtracted per ROI.
    sigma_vals : ndarray of shape (N,)
        Estimated noise standard deviation per ROI.
    mu_vals : ndarray of shape (N,)
        Estimated noise mean per ROI.
    A_vals : ndarray of shape (N,)
        Estimated noise amplitude per ROI (optional use).
    """
    T, N = deltaF_F.shape
    sigma_vals = np.zeros(N)
    mu_vals = np.zeros(N)
    A_vals = np.zeros(N)

    for i in range(N):
        trace = deltaF_F[:, i]
        sigma_vals[i], mu_vals[i], A_vals[i] = fit_function(trace)

    # Center the traces by removing estimated noise mean
    deltaF_centered = deltaF_F - mu_vals

    return deltaF_centered, sigma_vals, mu_vals, A_vals



In [ ]:
def estimate_and_center_noise_model(deltaF_F, fit_function=gaussfit_neg, verbose=True):
    """
    Fit Gaussian noise model for each ROI using the negative half of the ΔF/F distribution,
    then center each ROI's trace by subtracting the estimated mean (μ).

    This function is typically used after preprocessing (e.g., ΔF/F calculation)
    and helps prepare the data for statistical modeling or event detection.

    Parameters
    ----------
    deltaF_F : ndarray of shape (T, N)
        Preprocessed ΔF/F traces with T timepoints and N ROIs.
    fit_function : callable, optional
        Fitting function to use for noise modeling (default: gaussfit_neg).
        Should return (σ, μ, A, used_fallback) per ROI.
    verbose : bool
        Whether to print fitting statistics (e.g., number of fallback uses).

    Returns
    -------
    deltaF_centered : ndarray of shape (T, N)
        ΔF/F data with estimated μ subtracted per ROI.
    sigma_vals : ndarray of shape (N,)
        Estimated noise standard deviation (σ) per ROI.
    mu_vals : ndarray of shape (N,)
        Estimated Gaussian mean (μ) per ROI.
    A_vals : ndarray of shape (N,)
        Estimated Gaussian amplitude (A) per ROI.
    fallback_flags : list of bool
        List indicating whether each ROI required fallback estimation.
    """
    T, N = deltaF_F.shape
    sigma_vals = np.zeros(N)
    mu_vals = np.zeros(N)
    A_vals = np.zeros(N)
    fallback_flags = []

    # --- Fit noise model per ROI ---
    for i in range(N):
        trace = deltaF_F[:, i]
        sigma, mu, A, used_fallback = fit_function(trace)
        sigma_vals[i] = sigma
        mu_vals[i] = mu
        A_vals[i] = A
        fallback_flags.append(used_fallback)

    # --- Subtract estimated noise mean from each trace ---
    deltaF_centered = deltaF_F - mu_vals  # Broadcast subtraction across ROIs

    if verbose:
        n_fallback = sum(fallback_flags)
        print(f"[Noise Fit] Used fallback on {n_fallback} out of {N} ROIs "
              f"({100 * n_fallback / N:.1f}%)")

    return deltaF_centered, sigma_vals, mu_vals, A_vals, fallback_flags

deltaF_center, sigma_vals, mu_vals, A_vals, fallback_flags = estimate_and_center_noise_model(
    deltaF_F
)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Assume deltaF_F is (T, N), we need (N, T) for PCA
data = deltaF_center.T  # shape: (neurons, time)
pca = PCA(n_components=1)
scores = pca.fit_transform(data)

#Sort neurons by their score along the first PC
sort_idx = np.argsort(scores[:, 0])
sorted_data = deltaF_F[:, sort_idx].T  # shape: (neurons, time)

# Compute time vector in seconds
n_timepoints = sorted_data.shape[1]
time = np.arange(n_timepoints) / fps  # time axis in seconds

plt.figure(figsize=(10, 6))
im = plt.imshow(
    sorted_data,
    aspect='auto',
    cmap='gray_r',
    vmin=0, vmax=1,
    extent=[time[0], time[-1], 0, sorted_data.shape[0]]
)
plt.xlabel("Time (s)")
plt.ylabel("# Neuron (sorted)")
cbar = plt.colorbar(im, label="ΔF/F")
plt.title("ΔF/F Raster Plot (sorted by activity)")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from scipy.stats import gaussian_kde
from scipy.ndimage import gaussian_filter
from sklearn.neighbors import NearestNeighbors
from skimage.measure import label, regionprops

def normalize_dff(deltaF_center, sigma_vals):
    """
    Normalize ΔF/F traces by dividing each neuron trace by its estimated noise level (σ).

    Args:
        deltaF_center (ndarray): ΔF/F matrix, shape (T, N)
        sigma_vals (ndarray): Noise levels, shape (N,)

    Returns:
        ndarray: Normalized ΔF/F matrix
    """
    return deltaF_center / sigma_vals

def extract_transition_points(norm_data):
    """
    Create a 2D point cloud of transitions between ΔF/F(t) and ΔF/F(t+1) for all neurons.

    Args:
        norm_data (ndarray): Normalized ΔF/F, shape (T, N)

    Returns:
        ndarray: 2D points of transitions, shape (~T*N, 2)
    """
    x = norm_data[:-1, :].flatten()
    y = norm_data[1:, :].flatten()
    valid = ~np.isnan(x) & ~np.isnan(y)
    return np.vstack((x[valid], y[valid])).T

def estimate_kde_peak(points):
    """
    Estimate the peak (mode) of ΔF/F(t) and ΔF/F(t+1) using KDE.

    Args:
        points (ndarray): 2D transitions, shape (N, 2)

    Returns:
        tuple: (peak_x, peak_y)
    """
    kde_x = gaussian_kde(points[:, 0])
    kde_y = gaussian_kde(points[:, 1])
    xi = np.linspace(np.min(points), np.max(points), 200)
    peak_x = xi[np.argmax(kde_x(xi))]
    peak_y = xi[np.argmax(kde_y(xi))]
    return peak_x, peak_y

def generate_synthetic_noise(points, peak_x, peak_y):
    """
    Create synthetic noise points from the lower-left (negative) quadrant of the data,
    using a multivariate Gaussian with matching mean and covariance.

    Args:
        points (ndarray): Original transition points, shape (N, 2)
        peak_x (float): Estimated mode of ΔF/F(t)
        peak_y (float): Estimated mode of ΔF/F(t+1)

    Returns:
        ndarray: Synthetic noise points, shape (M, 2)
    """
    points_neg = points[(points[:, 0] < peak_x) & (points[:, 1] < peak_y)]
    mu_noise = [peak_x, peak_y]
    if len(points_neg) < 2:
        raise ValueError("Not enough points to compute covariance for noise model.")
    cov_noise = np.cov(points_neg, rowvar=False)
    synthetic_noise = np.random.multivariate_normal(mu_noise, cov_noise, size=4 * len(points_neg))
    return synthetic_noise

def create_grid(points, synthetic_noise, n_bins):
    """
    Create a shared 2D grid for histogramming data and synthetic noise.

    Args:
        points (ndarray): Real data points, shape (N, 2)
        synthetic_noise (ndarray): Synthetic noise points, shape (M, 2)
        n_bins (int): Number of bins for histogram in each dimension

    Returns:
        tuple: (grid, xev, yev)
    """
    combined = np.concatenate([points.flatten(), synthetic_noise.flatten()])
    vmin, vmax = np.floor(np.min(combined)), np.ceil(np.max(combined))
    grid = np.linspace(vmin, vmax, n_bins)
    xev, yev = np.meshgrid(grid, grid)
    return grid, xev, yev


def histogram2d(points, bins, grid):
    """
    Create a 2D histogram of transition points based on a shared grid.

    Args:
        points (ndarray): Points to bin, shape (N, 2)
        bins (int): Number of bins
        grid (ndarray): Bin edges

    Returns:
        ndarray: 2D histogram, shape (bins, bins)
    """
    xi = np.searchsorted(grid, points[:, 0]) - 1
    yi = np.searchsorted(grid, points[:, 1]) - 1
    xi = np.clip(xi, 0, bins - 1)
    yi = np.clip(yi, 0, bins - 1)
    hist = np.zeros((bins, bins))
    for i in range(len(xi)):
        hist[yi[i], xi[i]] += 1
    return hist

def compute_global_sigma(points, xev, yev, k_neighbors):
    """
    Estimate the global Gaussian smoothing sigma by computing average distances
    to the k-nearest real data points from each histogram bin location.

    Args:
        points (ndarray): Real data transition points, shape (N, 2)
        xev (ndarray): Meshgrid x-coordinates
        yev (ndarray): Meshgrid y-coordinates
        k_neighbors (int): Number of neighbors for density estimation

    Returns:
        float: Estimated sigma for Gaussian filter
    """
    grid_points = np.column_stack([xev.ravel(), yev.ravel()])
    nn = NearestNeighbors(n_neighbors=k_neighbors)
    nn.fit(points)
    dists = nn.kneighbors(grid_points, return_distance=True)[0]
    bin_spread = dists.mean(axis=1).reshape(xev.shape)
    return np.median(bin_spread)

def plot_density_comparison(density_data, density_noise, xev, yev, show=True):
    """
    Plot side-by-side heatmaps of real vs synthetic noise densities.

    Args:
        density_data (ndarray): Smoothed density of real ΔF/F transitions
        density_noise (ndarray): Smoothed density of synthetic noise
        xev (ndarray): Meshgrid x-coordinates
        yev (ndarray): Meshgrid y-coordinates
        show (bool): Whether to display the plots
    """
    if not show:
        return

    extent = [xev.min(), xev.max(), yev.min(), yev.max()]
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    im1 = axs[0].imshow(density_data.T, origin='lower', extent=extent, cmap='viridis', aspect='auto')
    axs[0].set_title("Real ΔF/F Transitions Density")
    axs[0].set_xlabel("ΔF/F(t)")
    axs[0].set_ylabel("ΔF/F(t+1)")
    plt.colorbar(im1, ax=axs[0], label='Density')

    im2 = axs[1].imshow(density_noise.T, origin='lower', extent=extent, cmap='viridis', aspect='auto')
    axs[1].set_title("Synthetic Noise Density")
    axs[1].set_xlabel("ΔF/F(t)")
    axs[1].set_ylabel("ΔF/F(t+1)")
    plt.colorbar(im2, ax=axs[1], label='Density')

    plt.tight_layout()
    plt.show()

def compute_significant_odds(points, density_data, density_noise, xev, yev=None, confCutOff=95, plotFlag=False):
    """
    Compute the significance mask where real transition density exceeds noise density by a confidence threshold.

    Args:
        points (ndarray): 2D transition points (ΔF/F(t), ΔF/F(t+1)), shape (M, 2)
        density_data (ndarray): Smoothed histogram of real transitions
        density_noise (ndarray): Smoothed histogram of synthetic noise transitions
        xev (ndarray): Meshgrid X-coordinates
        yev (ndarray, optional): Meshgrid Y-coordinates (for correct plotting)
        confCutOff (float): Confidence level for thresholding (default 95)
        plotFlag (bool): Whether to plot the significance map

    Returns:
        mapOfOdds (ndarray): Boolean mask (same shape as density_data) indicating significant bins
    """
    pCutOff = (100 - confCutOff) / 100.0
    mapOfOdds = density_noise <= (pCutOff * density_data)

    if plotFlag:
        if yev is None:
            yev = xev  # fallback if symmetric grid

        extent = [xev.min(), xev.max(), yev.min(), yev.max()]
        plt.figure(figsize=(6, 6))
        plt.plot(points[:, 0], points[:, 1], 'k.', alpha=0.2, markersize=1)
        plt.imshow(mapOfOdds.T, extent=extent, origin='lower', aspect='auto', alpha=0.6, cmap='Reds')
        plt.xlabel('z-transformed ΔF/F @ t', fontsize=14)
        plt.ylabel('z-transformed ΔF/F @ t+1', fontsize=14)
        plt.title(f'Significant Odds Mask (>{confCutOff}% confidence)')
        plt.axis('equal')
        plt.tight_layout()
        plt.show()

    return mapOfOdds

def rasterize_with_odds(norm_data, mapOfOdds, xev, yev, fps, tauDecay):
    """
    Rasterize ΔF/F activity using statistically significant transition maps,
    adapted from Romano et al. (2017).

    This function identifies significant neural transients by intersecting
    statistical significance (mapOfOdds) with biologically plausible rise/decay
    constraints, and uses 3-point temporal patterns to mark transients.

    Parameters
    ----------
    norm_data : ndarray, shape (T, N)
        Z-scored ΔF/F traces (i.e., ΔF/F divided by per-neuron sigma).
    mapOfOdds : ndarray of bool, shape (B, B)
        Boolean matrix indicating statistically significant transitions from ΔF/F(t) to ΔF/F(t+1).
    xev, yev : ndarray, shape (B, B)
        Meshgrids used for binning ΔF/F(t) and ΔF/F(t+1) values.
    fps : float
        Sampling rate (frames per second).
    tauDecay : float
        Expected calcium decay time constant (in seconds).

    Returns
    -------
    raster : ndarray of int, shape (T, N)
        Binary matrix indicating significant neural transients (1 = event).
    mapOfOddsJoint : ndarray of bool, shape (B, B)
        Refined joint transition map after applying baseline, decay, and rise filters.
    """
    T, N = norm_data.shape
    transfDataMatrix = norm_data

    # --- Step 1: Remove baseline noise blob (connected to 0,0) ---
    mask = ~mapOfOdds  # noise regions
    labeled = label(mask)  # label connected regions
    props = regionprops(labeled)
    indZero = np.argmax(xev[0, :] > 0)  # bin closest to ΔF/F = 0

    region_index = None
    for i, region in enumerate(props):
        if any(p[1] == indZero for p in region.coords):
            region_index = region.label
            break
    # Create corrected map: keep all transitions except baseline blob
    mapOfOddsCorrected = np.ones_like(mapOfOdds, dtype=bool)
    if region_index is not None:
        mapOfOddsCorrected[labeled == region_index] = 0

    # Step 2: Build decay map — exclude transitions faster than biologically plausible decay
    noiseBias = 1.0  # controls strictness of decay exclusion
    factorDecay = np.exp(-1 / (fps * tauDecay))  # expected decay factor per frame
    decayMap = np.ones_like(mapOfOdds, dtype=bool)
    for col in range(mapOfOdds.shape[1]):
        # Remove transitions where ΔF/F(t+1) decays too steeply from ΔF/F(t)
        mask = yev[:, 0] < factorDecay * (xev[0, col] - noiseBias) - noiseBias
        decayMap[mask, col] = 0

   # Step 3: Build rise map — remove extreme up transitions at top of the grid
    riseMap = np.ones_like(mapOfOdds, dtype=bool)
    riseMap[-1, :] = 0  # Discard transitions ending at the highest ΔF/F(t+1) bin

    # Step 4: Final joint significance map
    mapOfOddsJoint = mapOfOddsCorrected & decayMap & riseMap

    # Step 5: Raster generation
    raster = np.zeros_like(transfDataMatrix, dtype=int)
    bin_edges = xev[0, :]  # bin values used to digitize ΔF/F traces

    # For each neuron
    for n in range(N):
        trace = transfDataMatrix[:, n]
        bins = np.digitize(trace, bin_edges) - 1  # Convert ΔF/F values to bin indices

        for t in range(2, T - 2):  # Leave padding for 2-point transitions
            try:
                # Test three 3-point transition patterns
                optA = mapOfOddsJoint[bins[t + 1], bins[t]] and mapOfOddsJoint[bins[t], bins[t - 1]]
                optB = mapOfOddsJoint[bins[t], bins[t - 1]] and mapOfOddsJoint[bins[t - 1], bins[t - 2]]
                optC = mapOfOddsJoint[bins[t + 2], bins[t + 1]] and mapOfOddsJoint[bins[t + 1], bins[t]]
                if optA or optB or optC:
                    raster[t, n] = 1  # Mark as event
            except IndexError:
                continue  # Skip if bin index out of bounds

    return raster, mapOfOddsJoint


In [ ]:
def compute_noise_model_romano_fast_modular(
    deltaF_center,
    sigma_vals,
    n_bins=1000,
    k_neighbors=100,
    confCutOff=95,
    plot_odds=False,
    plot_density=False,
    fps=2.0,
    tauDecay=3.0,

):
    """
    Compute the Romano-style noise model and generate a raster of significant transients.

    This pipeline analyzes ΔF/F transitions using statistical and biophysical constraints
    to distinguish real neural events from noise.

    Parameters
    ----------
    deltaF_center : ndarray of shape (T, N)
        ΔF/F traces across time (T) and neurons (N).
    sigma_vals : ndarray of shape (N,)
        Estimated noise standard deviation per neuron.
    n_bins : int
        Number of bins for KDE/histogram grid.
    k_neighbors : int
        Number of nearest neighbors used to estimate smoothing scale.
    confCutOff : float
        Confidence level (in percent) used to threshold real vs noise transitions.
    plot_odds : bool
        If True, show significant transition mask.
    plot_density : bool
        If True, plot transition density of real vs synthetic noise.
    fps : float
        Imaging frame rate (Hz).
    tauDecay : float
        Calcium decay time constant (in seconds).

    Returns
    -------
    mapOfOdds : ndarray of shape (B, B)
        Binary map of significant transitions (real > noise).
    density_data : ndarray
        Smoothed density of real ΔF/F(t) → ΔF/F(t+1) transitions.
    density_noise : ndarray
        Smoothed density from synthetic Gaussian noise model.
    xev, yev : ndarray
        Meshgrid coordinates used for binning.
    """

    # --- Step 1: Normalize ΔF/F using per-neuron σ values
    norm_data = normalize_dff(deltaF_center, sigma_vals)

    # --- Step 2: Extract point cloud of ΔF/F(t), ΔF/F(t+1) transitions
    points = extract_transition_points(norm_data)

    # --- Step 3: Estimate the peak of KDE in each dimension to locate mode
    peak_x, peak_y = estimate_kde_peak(points)

    # --- Step 4: Generate synthetic noise from the negative quadrant
    # Modeled as a multivariate Gaussian fit to low activity region
    synthetic_noise = generate_synthetic_noise(points, peak_x, peak_y)

    # --- Step 5: Create a common 2D grid for histogramming real and noise data
    grid, xev, yev = create_grid(points, synthetic_noise, n_bins)

    # --- Step 6: Bin the real data and synthetic noise into histograms
    hist_data = histogram2d(points, n_bins, grid)
    hist_noise = histogram2d(synthetic_noise, n_bins, grid)

    # --- Step 7: Estimate adaptive smoothing scales via KNN on each distribution
    sigma_data = compute_global_sigma(points, xev, yev, k_neighbors)
    sigma_noise = compute_global_sigma(synthetic_noise, xev, yev, k_neighbors)

    # --- Step 8: Apply Gaussian smoothing to both histograms
    density_data = gaussian_filter(hist_data, sigma=sigma_data)
    density_noise = gaussian_filter(hist_noise, sigma=sigma_noise)

    # --- Step 9: Optionally show real vs noise density comparison
    if plot_density:
        plot_density_comparison(density_data, density_noise, xev, yev, show=True)

    # --- Step 10: Build a binary significance map (real density > noise threshold)
    mapOfOdds = compute_significant_odds(
        points,
        density_data,
        density_noise,
        xev,
        yev,
        confCutOff=confCutOff,
        plotFlag=plot_odds
    )

    # --- Step 11: Apply Romano rasterization with decay + rise constraints
    raster, mapOfOddsJoint = rasterize_with_odds(
        norm_data=norm_data,
        mapOfOdds=mapOfOdds,
        xev=xev,
        yev=yev,
        fps=fps,
        tauDecay=tauDecay
    )

    return mapOfOdds, density_data, density_noise, xev, yev, raster, mapOfOddsJoint


mapOfOdds, density_data, density_noise, xev, yev, raster, mapOfOddsJoint = compute_noise_model_romano_fast_modular(
    deltaF_center,
    sigma_vals,
    n_bins=1000,
    k_neighbors=100,
    confCutOff=95,
    plot_odds=False,
    plot_density=False,
    fps = 2.0,       # or your actual imaging rate
    tauDecay = 3.0,   # seconds (depends on calcium indicator, e.g. GCaMP6s ~ 1s)
)


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

def plot_raster_sorted_by_pca(raster_clean, fps=2):
    """
    Plots a binary raster (0/1) sorted by PCA on neural activity.

    Parameters:
        raster: (T, N) binary matrix of significant events
        fps: frames per second
    """
    data = raster_clean.T  # (neurons, time)
    #pca = PCA(n_components=1)
    #scores = pca.fit_transform(data)

    #sort_idx = np.argsort(scores[:, 0])
    sorted_raster = raster_clean[:, sort_idx].T  # (neurons, time)

    time = np.arange(sorted_raster.shape[1]) / fps

    plt.figure(figsize=(10, 6))
    im = plt.imshow(
        sorted_raster,
        aspect='auto',
        cmap='Greys',
        vmin=0, vmax=1,
        extent=[time[0], time[-1], 0, sorted_raster.shape[0]]
    )
    plt.xlabel("Time (s)")
    plt.ylabel("# Neuron (PCA sorted)")
    cbar = plt.colorbar(im, label="Activity (0/1)")
    plt.title("Raster Plot (neurons sorted by PCA)")
    plt.tight_layout()
    plt.show()

    return sort_idx

sort_idx = plot_raster_sorted_by_pca(raster, fps=2)


In [ ]:
# Assume deltaF_F is (T, N), we need (N, T) for PCA
data = deltaF_center.T  # shape: (neurons, time)
#pca = PCA(n_components=1)
#scores = pca.fit_transform(data)

#Sort neurons by their score along the first PC
#sort_idx = np.argsort(scores[:, 0])
sorted_data = deltaF_F[:, sort_idx].T  # shape: (neurons, time)

# Compute time vector in seconds
n_timepoints = sorted_data.shape[1]
time = np.arange(n_timepoints) / fps  # time axis in seconds

plt.figure(figsize=(10, 6))
im = plt.imshow(
    sorted_data,
    aspect='auto',
    cmap='gray_r',
    vmin=0, vmax=0.3,
    extent=[time[0], time[-1], 0, sorted_data.shape[0]]
)
plt.xlabel("Time (s)")
plt.ylabel("# Neuron (sorted)")
cbar = plt.colorbar(im, label="ΔF/F")
plt.title("ΔF/F Raster Plot (sorted by activity)")
plt.tight_layout()
plt.show()